# Real Data Example 1: Basic PSD Workflow with Co-60

This notebook demonstrates a complete PSD analysis workflow using **real Co-60 data**.

## About Co-60
- Cobalt-60 is a gamma emitter with two characteristic peaks:
  - 1173.2 keV
  - 1332.5 keV
- Useful for energy calibration and detector characterization
- Pure gamma source (no neutron contamination)

## What You'll Learn
1. Load real waveform data from CSV
2. Quality control and event validation
3. Calculate PSD parameters
4. Visualize waveforms and PSD distributions
5. Basic gamma spectroscopy

## 1. Setup and Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add parent directory to path
sys.path.insert(0, '..')

from psd_analysis import (
    load_psd_data,
    validate_events,
    calculate_psd_ratio,
    find_peaks_in_spectrum
)
from psd_analysis.visualization import plot_psd_scatter, plot_energy_spectra

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful")

## 2. Load Real Co-60 Data

The data format is:
```
BOARD;CHANNEL;TIMETAG;ENERGY;ENERGYSHORT;FLAGS;PROBE_CODE;SAMPLES
```
where SAMPLES contains 368 waveform samples per event.

In [ ]:
# Load Co-60 data
df = load_psd_data('../data/raw/co60_sample.csv')

print(f"\n✓ Loaded {len(df)} events")
print(f"\nData columns:")
print(f"  Metadata: BOARD, CHANNEL, TIMETAG, ENERGY, ENERGYSHORT, FLAGS, PROBE_CODE")
print(f"  Waveforms: {df.shape[1] - 7} samples per event")

print(f"\nFirst few events:")
print(df[['ENERGY', 'ENERGYSHORT', 'FLAGS']].head())

## 3. Visualize Raw Waveforms

Let's look at the actual waveform shapes from the detector.

In [ ]:
# Extract waveform samples
sample_cols = [col for col in df.columns if col.startswith('SAMPLE_')]
waveform_0 = df[sample_cols].iloc[0].values
waveform_1 = df[sample_cols].iloc[1].values

# Time axis (4 ns per sample at 250 MHz)
time_ns = np.arange(len(waveform_0)) * 4

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Event 0
ax = axes[0]
ax.plot(time_ns, waveform_0, 'b-', linewidth=1.5, alpha=0.8)
ax.axhline(np.mean(waveform_0[:50]), color='red', linestyle='--', 
           label=f'Baseline: {np.mean(waveform_0[:50]):.0f} ADC', linewidth=2)
ax.set_xlabel('Time (ns)', fontsize=12, fontweight='bold')
ax.set_ylabel('ADC Value', fontsize=12, fontweight='bold')
ax.set_title(f'Event 0: E={df["ENERGY"].iloc[0]} ADC, E_short={df["ENERGYSHORT"].iloc[0]} ADC',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Event 1
ax = axes[1]
ax.plot(time_ns, waveform_1, 'g-', linewidth=1.5, alpha=0.8)
ax.axhline(np.mean(waveform_1[:50]), color='red', linestyle='--',
           label=f'Baseline: {np.mean(waveform_1[:50]):.0f} ADC', linewidth=2)
ax.set_xlabel('Time (ns)', fontsize=12, fontweight='bold')
ax.set_ylabel('ADC Value', fontsize=12, fontweight='bold')
ax.set_title(f'Event 1: E={df["ENERGY"].iloc[1]} ADC, E_short={df["ENERGYSHORT"].iloc[1]} ADC',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Raw waveforms plotted")
print("\nNote: Negative-going pulses are typical for PMT signals")

## 4. Calculate PSD Parameter

PSD = (ENERGY - ENERGYSHORT) / ENERGY

For gamma rays, we expect low PSD values since gammas produce less slow scintillation light.

In [ ]:
# Calculate PSD
df = calculate_psd_ratio(df)

print("\nPSD Statistics:")
print(f"  Mean PSD: {df['PSD'].mean():.4f}")
print(f"  Std PSD:  {df['PSD'].std():.4f}")
print(f"  Range:    [{df['PSD'].min():.4f}, {df['PSD'].max():.4f}]")

print("\nPer-event PSD values:")
for i, row in df.iterrows():
    print(f"  Event {i}: E={row['ENERGY']:4d}, E_short={row['ENERGYSHORT']:3d}, PSD={row['PSD']:.4f}")

print("\n✓ These low PSD values are expected for gamma rays!")

## 5. Energy Spectrum

For real Co-60 data with more events, we would see two distinct peaks at the characteristic gamma energies.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Plot energy distribution
ax.hist(df['ENERGY'], bins=50, alpha=0.7, edgecolor='black', linewidth=1.2)
ax.set_xlabel('Energy (ADC)', fontsize=13, fontweight='bold')
ax.set_ylabel('Counts', fontsize=13, fontweight='bold')
ax.set_title('Co-60 Energy Spectrum (Uncalibrated)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Mark the two events
for i, energy in enumerate(df['ENERGY']):
    ax.axvline(energy, color='red', linestyle='--', alpha=0.5,
               label=f'Event {i}: {energy} ADC' if i < 2 else None)

ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

print("\n✓ With more events, you would see two distinct peaks corresponding to:")
print("   - 1173.2 keV gamma ray")
print("   - 1332.5 keV gamma ray")

## 6. PSD vs Energy Scatter Plot

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))

# Scatter plot
ax.scatter(df['ENERGY'], df['PSD'], s=100, c='blue', alpha=0.7,
           edgecolors='black', linewidth=2, label='Co-60 gamma events')

ax.set_xlabel('Energy (ADC)', fontsize=13, fontweight='bold')
ax.set_ylabel('PSD Parameter', fontsize=13, fontweight='bold')
ax.set_title('PSD vs Energy - Co-60 Source', fontsize=14, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

# Add reference region for gamma rays
ax.axhspan(0, 0.15, alpha=0.1, color='blue', label='Typical gamma region')
ax.text(df['ENERGY'].mean(), 0.14, 'Gamma Region', 
        ha='center', fontsize=11, color='blue', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ PSD scatter plot shows gamma-ray signature")
print("   (Low PSD values indicate gamma rays)")

## Summary

In this notebook, we demonstrated:

✅ **Data Loading**: Successfully loaded real waveform data from CSV
- Parsed 368 waveform samples per event
- Extracted ENERGY and ENERGYSHORT parameters

✅ **Waveform Visualization**: Plotted raw detector pulses
- Observed negative-going PMT signals
- Identified baseline and pulse characteristics

✅ **PSD Calculation**: Computed tail-to-total ratio
- Low PSD values (~0.062-0.057) confirm gamma-ray events
- Expected for Co-60 (pure gamma source)

✅ **Energy Analysis**: Examined uncalibrated spectrum
- Two events with different energies observed
- Ready for energy calibration in next notebook

## Next Steps

Continue to **Real Data Example 2** for:
- Energy calibration using Co-60's known peak energies
- Converting ADC values to keV
- Peak finding and Gaussian fitting
- Energy resolution calculation